# Running Reducers Locally

This document serves as a walkthrough to setting up a reducer and running it locally.

## Initial Setup

The Learning Observer platform offers an offline mode that will initialize all the necessary settings and modules.

In [1]:
import learning_observer.offline
learning_observer.offline.init()

WARN:: Unrecognized Minty URL detected: https://cdn.jsdelivr.net/npm/bootswatch@5.2.3/dist/minty/bootstrap.min.css
You will need to update dash bootstrap components hash value.

WARN:: Unrecognized Minty URL detected: https://cdn.jsdelivr.net/npm/bootswatch@5.2.3/dist/minty/bootstrap.min.css
You will need to update dash bootstrap components hash value.

WARN:: Unrecognized Minty URL detected: https://cdn.jsdelivr.net/npm/bootswatch@5.2.3/dist/minty/bootstrap.min.css
You will need to update dash bootstrap components hash value.



## Creating the Reducer

To create turn a function into a reducer, we need to wrap it in the `kvs_pipeline` decorator. This decorator handles setting the appropriate items in the KVS when the reducer is ran. Note that reducer functions are expected to take in an `event` and a `state` parameter. The function should output the `Internal` and `External` state.

Additionally, we need to register our new reducer with the Learning Observer platform. When we want to query the communication protocol for data from this reducer, the system looks for the `id` of our reducer in the registered reducers.

In [2]:
from learning_observer.stream_analytics.helpers import kvs_pipeline, KeyField, Scope
import learning_observer.module_loader

# create reducer function
@kvs_pipeline(scope=Scope([KeyField.STUDENT]), module_override='testing')
async def event_counter(event, state):
    if state is None:
        state = {}
    state['event_count'] = state.get('event_count', 0) + 1
    return state, state

# define specific information and register reducer
reducer = {
    'context': 'local.testing',
    'function': event_counter,
    'scope': Scope([KeyField.STUDENT]),
    'default': {'event_count': 0},
    'module': 'testing',
    'id': 'test-event-reducer'
}
reducers = learning_observer.module_loader.add_reducer(reducer, 'test-event-reducer')

## Running the Reducer over Data

Learning Observer's offline mode allows for processing event files through reducers. First, we define which files we want ran. Then, we process them through a specific reducer (the `pipeline` parameter`).

In [3]:
import os
input_path = os.path.join(os.getcwd(), 'learning_observer', 'learning_observer', 'logs', 'sample01.log')
await learning_observer.offline.process_file(file_path=input_path, source="localhost.testcase", pipeline=event_counter, userid='Tester')

(1214, 'localhost.testcase', 'Tester')

In [4]:
# check to see if our reducer ran correctly
kvs = learning_observer.kvs.KVS()
print("Keys:")
keys = await kvs.keys()
print(keys)
await kvs['Internal,testing.event_counter,STUDENT:Tester']


Keys:
['Internal,testing.event_counter,STUDENT:Tester', 'External,testing.event_counter,STUDENT:Tester']


{'event_count': 1214}

## Query Reducer

To properly query the results of a reducer in the communication protocol, we need to create and execute an execution DAG.

In [5]:
import learning_observer.communication_protocol.query as q
course_roster = q.call('learning_observer.courseroster')
EXECUTION_DAG = {
    "execution_dag": {
        "roster": course_roster(runtime=q.parameter("runtime"), course_id=q.parameter("course_id", required=True)),
        'event_count': q.select(q.keys('test-event-reducer', STUDENTS=q.variable("roster"), STUDENTS_path='user_id'), fields={'event_count': 'event_count'}),
    },
    "exports": {
        'event_count': {
            'returns': 'event_count'
        }
    }
}

In [6]:
import learning_observer.communication_protocol.integration
import learning_observer.runtime
func = learning_observer.communication_protocol.integration.prepare_dag_execution(EXECUTION_DAG, ['event_count'])
await func(course_id=12345, runtime=learning_observer.runtime.Runtime(None))

{'event_count': [{'event_count': 1214}]}